In [8]:
import numpy as np                                  # type: ignore
from skimage.measure import label, regionprops      # type: ignore
from skimage.io import imread                       # type: ignore
from skimage.filters import threshold_yen           # type: ignore
import glob
from dataclasses import dataclass
import napari
import os

In [12]:
def image_path(exp, time, slice, win=False):
    if win:
        path = 'Z:/Reconstructions/' + exp
    else:
        path = '../MasterThesisData/' + exp
    folder_name = 'entry' + str(time).zfill(4) + '_no_extpag_db0100_vol'
    image_name = 'entry' + str(time).zfill(4) + '_no_extpag_db0100_vol_' + str(slice).zfill(6) + '.tiff'
    return os.path.join(path, folder_name, image_name)

def read_sequence2(exp, time=0, slice=0, first_time=0, last_time=220, first_slice=20, last_slice=260, volume=True, win=False):

    if volume:
        test_image = imread(image_path(exp, time, first_slice, win))
        sequence = np.zeros((last_slice-first_slice, test_image.shape[0], test_image.shape[1]))
        for slice in range(first_slice, last_slice):
            image = imread(image_path(exp, time, slice, win))
            sequence[slice-first_slice,:,:] = image
    else:
        test_image = imread(image_path(exp, first_time, slice, win))
        sequence = np.zeros((last_time-first_time, test_image.shape[0], test_image.shape[1]))
        for time in range(first_time, last_time):
            image = imread(image_path(exp, time, slice, win))
            sequence[time-first_time,:,:] = image

    return sequence

Volume check

In [13]:
sequence = read_sequence2('P28A_FT_H_Exp1', time=150, win=True)

Time evolution check

In [15]:
sequence = read_sequence2('P28A_FT_H_Exp1', slice=150, volume=False, win=True)

In [16]:
viewer = napari.Viewer()
_ = viewer.add_image(sequence)